In [ ]:
import ase.io as ase_io
import matplotlib.pyplot as plt

from e3nn.io import CartesianTensor

from soprano.properties import nmr
from soprano.properties.nmr import MSTensor

import torch
import numpy as np

In [ ]:
ct_symm = CartesianTensor("ij=ji")
ct_antisymm = CartesianTensor("ij=-ji")

In [ ]:
def get_contrib(frm, tag):

    frame = frm.copy()
    frame.arrays["ms"] = frame.arrays[tag].reshape(-1, 3, 3)
    magres = frame.arrays["ms"]
    l1 = ct_antisymm.from_cartesian(torch.tensor(magres)).numpy()

    symm = ct_symm.from_cartesian(torch.tensor(magres)).numpy()
    l0 = symm[:, 0]
    l2 = symm[:, 1:]

    aniso = nmr.MSAnisotropy.get(frame)
    asymm = nmr.MSAsymmetry.get(frame)

    skew = nmr.MSSkew.get(frame)
    span = nmr.MSSpan.get(frame)

    euler = []
    ms_tensor = MSTensor.get(frame)
    for t in ms_tensor:
        equivalent_euler = t.equivalent_euler_angles("zyz", passive=True)
        euler.append(equivalent_euler)

    return l0, l1, l2, aniso, asymm, skew, span, euler

In [ ]:
def get_rmse(a, b, perc=True):
    rmse = np.sqrt(np.mean((a - b) ** 2))
    if perc:
        return 100 * rmse / b.std(ddof=1)
    return rmse


def get_rmse_euler(ml_all, dft_all):

    r = []
    p = []
    for ml, dft in zip(ml_all, dft_all):
        for j in range(len(ml)):
            da = np.abs(dft[j][:, 0][:, np.newaxis] - ml[j][:, 0])
            db = np.abs(dft[j][:, 1][:, np.newaxis] - ml[j][:, 1])
            dc = np.abs(dft[j][:, 2][:, np.newaxis] - ml[j][:, 2])
            r.append(
                [
                    dft[j][:, 0][np.where(np.isclose(da, da.min()))[0]].min(),
                    dft[j][:, 1][np.where(np.isclose(db, db.min()))[0]].min(),
                    dft[j][:, 2][np.where(np.isclose(dc, dc.min()))[0]].min(),
                ]
            )
            p.append(
                [
                    ml[j][:, 0][np.where(np.isclose(da, da.min()))[1]].min(),
                    ml[j][:, 1][np.where(np.isclose(db, db.min()))[1]].min(),
                    ml[j][:, 2][np.where(np.isclose(dc, dc.min()))[1]].min(),
                ]
            )
    r = np.array(r)
    p = np.array(p)
    return [
        get_rmse(p[:, 0], r[:, 0]),
        get_rmse(p[:, 1], r[:, 1]),
        get_rmse(p[:, 2], r[:, 2]),
    ]

In [ ]:
frames = ase_io.read("../data/train_test/test_with_all_labels.xyz", ":")

In [ ]:
list(frames[0].arrays.keys())

In [ ]:
qm_l0 = []
qm_l1 = []
qm_l2 = []
qm_aniso = []
qm_asymm = []
qm_skew = []
qm_span = []
qm_euler = []

for frm in frames:
    res = get_contrib(frm, "QM_ms")
    qm_l0.append(res[0])
    qm_l1.append(res[1])
    qm_l2.append(res[2])
    qm_aniso.append(res[3])
    qm_asymm.append(res[4])
    qm_skew.append(res[5])
    qm_span.append(res[6])
    qm_euler.append(res[7])

qm_l0 = np.array(qm_l0)
qm_l1 = np.array(qm_l1)
qm_l2 = np.array(qm_l2)
qm_aniso = np.array(qm_aniso)
qm_asymm = np.array(qm_asymm)
qm_skew = np.array(qm_skew)
qm_span = np.array(qm_span)
qm_euler = np.array(qm_euler)

In [ ]:
ml_tp_single_l0 = []
ml_tp_single_l1 = []
ml_tp_single_l2 = []
ml_tp_single_aniso = []
ml_tp_single_asymm = []
ml_tp_single_skew = []
ml_tp_single_span = []
ml_tp_single_euler = []

for t in [
    "ML_TP_single_1o_ms",
    "ML_TP_single_2e_ms",
    "ML_TP_single_3o_ms",
    "ML_TP_single_4e_ms",
]:
    for frm in frames:
        res = get_contrib(frm, t)
        ml_tp_single_l0.append(res[0])
        ml_tp_single_l1.append(res[1])
        ml_tp_single_l2.append(res[2])
        ml_tp_single_aniso.append(res[3])
        ml_tp_single_asymm.append(res[4])
        ml_tp_single_skew.append(res[5])
        ml_tp_single_span.append(res[6])
        ml_tp_single_euler.append(res[7])

ml_tp_single_l0 = np.array(ml_tp_single_l0).reshape(4, -1, 144)
ml_tp_single_l1 = np.array(ml_tp_single_l1).reshape(4, -1, 144, 3)
ml_tp_single_l2 = np.array(ml_tp_single_l2).reshape(4, -1, 144, 5)
ml_tp_single_aniso = np.array(ml_tp_single_aniso).reshape(4, -1, 144)
ml_tp_single_asymm = np.array(ml_tp_single_asymm).reshape(4, -1, 144)
ml_tp_single_skew = np.array(ml_tp_single_skew).reshape(4, -1, 144)
ml_tp_single_span = np.array(ml_tp_single_span).reshape(4, -1, 144)
ml_tp_single_euler = np.array(ml_tp_single_euler).reshape(4, -1, 144, 4, 3)

In [ ]:
ml_tp_64_l0 = []
ml_tp_64_l1 = []
ml_tp_64_l2 = []
ml_tp_64_aniso = []
ml_tp_64_asymm = []
ml_tp_64_skew = []
ml_tp_64_span = []
ml_tp_64_euler = []

for t in [
    "ML_TP_64_1o_ms",
    "ML_TP_64_2e_ms",
    "ML_TP_64_3o_ms",
    "ML_TP_64_4e_ms",
]:
    for frm in frames:
        res = get_contrib(frm, t)
        ml_tp_64_l0.append(res[0])
        ml_tp_64_l1.append(res[1])
        ml_tp_64_l2.append(res[2])
        ml_tp_64_aniso.append(res[3])
        ml_tp_64_asymm.append(res[4])
        ml_tp_64_skew.append(res[5])
        ml_tp_64_span.append(res[6])
        ml_tp_64_euler.append(res[7])

ml_tp_64_l0 = np.array(ml_tp_64_l0).reshape(4, -1, 144)
ml_tp_64_l1 = np.array(ml_tp_64_l1).reshape(4, -1, 144, 3)
ml_tp_64_l2 = np.array(ml_tp_64_l2).reshape(4, -1, 144, 5)
ml_tp_64_aniso = np.array(ml_tp_64_aniso).reshape(4, -1, 144)
ml_tp_64_asymm = np.array(ml_tp_64_asymm).reshape(4, -1, 144)
ml_tp_64_skew = np.array(ml_tp_64_skew).reshape(4, -1, 144)
ml_tp_64_span = np.array(ml_tp_64_span).reshape(4, -1, 144)
ml_tp_64_euler = np.array(ml_tp_64_euler).reshape(4, -1, 144, 4, 3)

In [ ]:
ml_tp_4096_l0 = []
ml_tp_4096_l1 = []
ml_tp_4096_l2 = []
ml_tp_4096_aniso = []
ml_tp_4096_asymm = []
ml_tp_4096_skew = []
ml_tp_4096_span = []
ml_tp_4096_euler = []

for t in [
    "ML_TP_4096_1o_ms",
    "ML_TP_4096_2e_ms",
    "ML_TP_4096_3o_ms",
    "ML_TP_4096_4e_ms",
]:
    for frm in frames:
        res = get_contrib(frm, t)
        ml_tp_4096_l0.append(res[0])
        ml_tp_4096_l1.append(res[1])
        ml_tp_4096_l2.append(res[2])
        ml_tp_4096_aniso.append(res[3])
        ml_tp_4096_asymm.append(res[4])
        ml_tp_4096_skew.append(res[5])
        ml_tp_4096_span.append(res[6])
        ml_tp_4096_euler.append(res[7])

ml_tp_4096_l0 = np.array(ml_tp_4096_l0).reshape(4, -1, 144)
ml_tp_4096_l1 = np.array(ml_tp_4096_l1).reshape(4, -1, 144, 3)
ml_tp_4096_l2 = np.array(ml_tp_4096_l2).reshape(4, -1, 144, 5)
ml_tp_4096_aniso = np.array(ml_tp_4096_aniso).reshape(4, -1, 144)
ml_tp_4096_asymm = np.array(ml_tp_4096_asymm).reshape(4, -1, 144)
ml_tp_4096_skew = np.array(ml_tp_4096_skew).reshape(4, -1, 144)
ml_tp_4096_span = np.array(ml_tp_4096_span).reshape(4, -1, 144)
ml_tp_4096_euler = np.array(ml_tp_4096_euler).reshape(4, -1, 144, 4, 3)

In [ ]:
si_idx = frames[0].numbers == 14
o_idx = frames[0].numbers == 8

In [ ]:
euler_rmse = np.array(
    [get_rmse_euler(x[:, o_idx], qm_euler[:, o_idx]) for x in ml_tp_64_euler]
)
o_err_ml_tp_64_alpha = euler_rmse[:, 0]
o_err_ml_tp_64_beta = euler_rmse[:, 1]
o_err_ml_tp_64_gamma = euler_rmse[:, 2]

In [ ]:
si_err_ml_tp_single_l0 = [
    get_rmse(x[:, si_idx], qm_l0[:, si_idx]) for x in ml_tp_single_l0
]
si_err_ml_tp_single_l1 = [
    get_rmse(x[:, si_idx], qm_l1[:, si_idx]) for x in ml_tp_single_l1
]
si_err_ml_tp_single_l2 = [
    get_rmse(x[:, si_idx], qm_l2[:, si_idx]) for x in ml_tp_single_l2
]
si_err_ml_tp_single_aniso = [
    get_rmse(x[:, si_idx], qm_aniso[:, si_idx]) for x in ml_tp_single_aniso
]
si_err_ml_tp_single_asymm = [
    get_rmse(x[:, si_idx], qm_asymm[:, si_idx]) for x in ml_tp_single_asymm
]
si_err_ml_tp_single_skew = [
    get_rmse(x[:, si_idx], qm_skew[:, si_idx]) for x in ml_tp_single_skew
]
si_err_ml_tp_single_span = [
    get_rmse(x[:, si_idx], qm_span[:, si_idx]) for x in ml_tp_single_span
]
euler_rmse = np.array(
    [get_rmse_euler(x[:, si_idx], qm_euler[:, si_idx]) for x in ml_tp_single_euler]
)
si_err_ml_tp_single_alpha = euler_rmse[:, 0]
si_err_ml_tp_single_beta = euler_rmse[:, 1]
si_err_ml_tp_single_gamma = euler_rmse[:, 2]

o_err_ml_tp_single_l0 = [
    get_rmse(x[:, o_idx], qm_l0[:, o_idx]) for x in ml_tp_single_l0
]
o_err_ml_tp_single_l1 = [
    get_rmse(x[:, o_idx], qm_l1[:, o_idx]) for x in ml_tp_single_l1
]
o_err_ml_tp_single_l2 = [
    get_rmse(x[:, o_idx], qm_l2[:, o_idx]) for x in ml_tp_single_l2
]
o_err_ml_tp_single_aniso = [
    get_rmse(x[:, o_idx], qm_aniso[:, o_idx]) for x in ml_tp_single_aniso
]
o_err_ml_tp_single_asymm = [
    get_rmse(x[:, o_idx], qm_asymm[:, o_idx]) for x in ml_tp_single_asymm
]
o_err_ml_tp_single_skew = [
    get_rmse(x[:, o_idx], qm_skew[:, o_idx]) for x in ml_tp_single_skew
]
o_err_ml_tp_single_span = [
    get_rmse(x[:, o_idx], qm_span[:, o_idx]) for x in ml_tp_single_span
]
euler_rmse = np.array(
    [get_rmse_euler(x[:, o_idx], qm_euler[:, o_idx]) for x in ml_tp_single_euler]
)
o_err_ml_tp_single_alpha = euler_rmse[:, 0]
o_err_ml_tp_single_beta = euler_rmse[:, 1]
o_err_ml_tp_single_gamma = euler_rmse[:, 2]

In [ ]:
si_err_ml_tp_64_l0 = [get_rmse(x[:, si_idx], qm_l0[:, si_idx]) for x in ml_tp_64_l0]
si_err_ml_tp_64_l1 = [get_rmse(x[:, si_idx], qm_l1[:, si_idx]) for x in ml_tp_64_l1]
si_err_ml_tp_64_l2 = [get_rmse(x[:, si_idx], qm_l2[:, si_idx]) for x in ml_tp_64_l2]
si_err_ml_tp_64_aniso = [
    get_rmse(x[:, si_idx], qm_aniso[:, si_idx]) for x in ml_tp_64_aniso
]
si_err_ml_tp_64_asymm = [
    get_rmse(x[:, si_idx], qm_asymm[:, si_idx]) for x in ml_tp_64_asymm
]
si_err_ml_tp_64_skew = [
    get_rmse(x[:, si_idx], qm_skew[:, si_idx]) for x in ml_tp_64_skew
]
si_err_ml_tp_64_span = [
    get_rmse(x[:, si_idx], qm_span[:, si_idx]) for x in ml_tp_64_span
]
euler_rmse = np.array(
    [get_rmse_euler(x[:, si_idx], qm_euler[:, si_idx]) for x in ml_tp_64_euler]
)
si_err_ml_tp_64_alpha = euler_rmse[:, 0]
si_err_ml_tp_64_beta = euler_rmse[:, 1]
si_err_ml_tp_64_gamma = euler_rmse[:, 2]


o_err_ml_tp_64_l0 = [get_rmse(x[:, o_idx], qm_l0[:, o_idx]) for x in ml_tp_64_l0]
o_err_ml_tp_64_l1 = [get_rmse(x[:, o_idx], qm_l1[:, o_idx]) for x in ml_tp_64_l1]
o_err_ml_tp_64_l2 = [get_rmse(x[:, o_idx], qm_l2[:, o_idx]) for x in ml_tp_64_l2]
o_err_ml_tp_64_aniso = [
    get_rmse(x[:, o_idx], qm_aniso[:, o_idx]) for x in ml_tp_64_aniso
]
o_err_ml_tp_64_asymm = [
    get_rmse(x[:, o_idx], qm_asymm[:, o_idx]) for x in ml_tp_64_asymm
]
o_err_ml_tp_64_skew = [get_rmse(x[:, o_idx], qm_skew[:, o_idx]) for x in ml_tp_64_skew]
o_err_ml_tp_64_span = [get_rmse(x[:, o_idx], qm_span[:, o_idx]) for x in ml_tp_64_span]
euler_rmse = np.array(
    [get_rmse_euler(x[:, o_idx], qm_euler[:, o_idx]) for x in ml_tp_64_euler]
)
o_err_ml_tp_64_alpha = euler_rmse[:, 0]
o_err_ml_tp_64_beta = euler_rmse[:, 1]
o_err_ml_tp_64_gamma = euler_rmse[:, 2]

In [ ]:
si_err_ml_tp_4096_l0 = [get_rmse(x[:, si_idx], qm_l0[:, si_idx]) for x in ml_tp_4096_l0]
si_err_ml_tp_4096_l1 = [get_rmse(x[:, si_idx], qm_l1[:, si_idx]) for x in ml_tp_4096_l1]
si_err_ml_tp_4096_l2 = [get_rmse(x[:, si_idx], qm_l2[:, si_idx]) for x in ml_tp_4096_l2]
si_err_ml_tp_4096_aniso = [
    get_rmse(x[:, si_idx], qm_aniso[:, si_idx]) for x in ml_tp_4096_aniso
]
si_err_ml_tp_4096_asymm = [
    get_rmse(x[:, si_idx], qm_asymm[:, si_idx]) for x in ml_tp_4096_asymm
]
si_err_ml_tp_4096_skew = [
    get_rmse(x[:, si_idx], qm_skew[:, si_idx]) for x in ml_tp_4096_skew
]
si_err_ml_tp_4096_span = [
    get_rmse(x[:, si_idx], qm_span[:, si_idx]) for x in ml_tp_4096_span
]
euler_rmse = np.array(
    [get_rmse_euler(x[:, si_idx], qm_euler[:, si_idx]) for x in ml_tp_4096_euler]
)
si_err_ml_tp_4096_alpha = euler_rmse[:, 0]
si_err_ml_tp_4096_beta = euler_rmse[:, 1]
si_err_ml_tp_4096_gamma = euler_rmse[:, 2]


o_err_ml_tp_4096_l0 = [get_rmse(x[:, o_idx], qm_l0[:, o_idx]) for x in ml_tp_4096_l0]
o_err_ml_tp_4096_l1 = [get_rmse(x[:, o_idx], qm_l1[:, o_idx]) for x in ml_tp_4096_l1]
o_err_ml_tp_4096_l2 = [get_rmse(x[:, o_idx], qm_l2[:, o_idx]) for x in ml_tp_4096_l2]
o_err_ml_tp_4096_aniso = [
    get_rmse(x[:, o_idx], qm_aniso[:, o_idx]) for x in ml_tp_4096_aniso
]
o_err_ml_tp_4096_asymm = [
    get_rmse(x[:, o_idx], qm_asymm[:, o_idx]) for x in ml_tp_4096_asymm
]
o_err_ml_tp_4096_skew = [
    get_rmse(x[:, o_idx], qm_skew[:, o_idx]) for x in ml_tp_4096_skew
]
o_err_ml_tp_4096_span = [
    get_rmse(x[:, o_idx], qm_span[:, o_idx]) for x in ml_tp_4096_span
]
euler_rmse = np.array(
    [get_rmse_euler(x[:, o_idx], qm_euler[:, o_idx]) for x in ml_tp_4096_euler]
)
o_err_ml_tp_4096_alpha = euler_rmse[:, 0]
o_err_ml_tp_4096_beta = euler_rmse[:, 1]
o_err_ml_tp_4096_gamma = euler_rmse[:, 2]

In [ ]:
fig = plt.figure(figsize=(7.3, 3.5))
step = 6
ministep = 1.5

xlabels = [
    "\n\n$\sigma^{(0)}$",
    "\n\n$\sigma^{(1)}$",
    "\n\n$\sigma^{(2)}$",
    "\n\n$\zeta_\sigma$",
    "\n\n$\eta_\sigma$",
    "\n\n$\kappa_\sigma$",
    "\n\n$\Omega_\sigma$",
    "\n\n$\\alpha_\sigma$",
    "\n\n$\\beta_\sigma$",
    "\n\n$\\gamma_\sigma$",
]

ax = fig.add_subplot(211)
ax.set_yscale("log")
ax.set_ylim(6, 650)


############# plot the Silicon MS single TP ############
xticks = []
x = np.arange(4)
ax.plot(x, si_err_ml_tp_single_l0, "C0.--", lw=0.75, label="1 tensor product")
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_l1, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_l2, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_aniso, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_asymm, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_skew, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_span, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_alpha, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_beta, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_single_gamma, "C0.--", lw=0.75)


############# plot th-e Silicon MS 64 TPs ############
xticks = []
x = np.arange(4)
ax.plot(x, si_err_ml_tp_64_l0, "C0d--", lw=0.75, ms=4, label="64 tensor product")
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_l1, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_l2, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_aniso, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_asymm, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_skew, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_span, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_alpha, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_beta, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_64_gamma, "C0d--", lw=0.75, ms=4)


############# plot th-e Silicon MS 4096 TP ############
xticks = []
x = np.arange(4)
ax.plot(x, si_err_ml_tp_4096_l0, "C0.-", lw=0.75, label="4096 tensor product")
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_l1, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_l2, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_aniso, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_asymm, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_skew, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_span, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_alpha, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_beta, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, si_err_ml_tp_4096_gamma, "C0.-", lw=0.75)

xticks += x.tolist()

ax.set_xticks(np.hstack(xticks))
ax.set_xticklabels(())

ax.set_ylabel("%RMSE", fontsize=9)
for i, x in enumerate(xlabels):
    ax.text(x=xticks[1::4][i], y=190, s=x, fontsize=9)
ax.text(0, 400, "silicon", fontsize=9)
ax.grid(lw=0.5, alpha=0.5, which="both")
ax.tick_params(axis="both", labelsize=7)
ax.set_xlim(xticks[0] - 1, xticks[-1] + 1)

l, h = ax.get_legend_handles_labels()
ax.legend(
    l,
    h,
    loc="upper center",
    fontsize=9,
    bbox_to_anchor=(0.5, 1.3),
    ncols=3,
    frameon=False,
)


ax = fig.add_subplot(212)
ax.set_yscale("log")

############# plot the Oxygen MS single TP ############
xticks = []
x = np.arange(4)
ax.plot(x, o_err_ml_tp_single_l0, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_l1, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_l2, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_aniso, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_asymm, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_skew, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_span, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_alpha, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_beta, "C0.--", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_single_gamma, "C0.--", lw=0.75)


############# plot the Oxygen MS 64 TPs ############
xticks = []
x = np.arange(4)
ax.plot(x, o_err_ml_tp_64_l0, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_l1, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_l2, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_aniso, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_asymm, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_skew, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_span, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_alpha, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_beta, "C0d--", lw=0.75, ms=4)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_64_gamma, "C0d--", lw=0.75, ms=4)


############# plot the Oxygen MS 4096 TP ############
xticks = []
x = np.arange(4)
ax.plot(x, o_err_ml_tp_4096_l0, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_l1, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_l2, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_aniso, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_asymm, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_skew, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_span, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_alpha, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_beta, "C0.-", lw=0.75)
xticks += x.tolist()
ax.vlines(xticks[-1] + ministep, 1, 200, color="k", ls="--", lw=0.75)

x += step
ax.plot(x, o_err_ml_tp_4096_gamma, "C0.-", lw=0.75)

xticks += x.tolist()

ax.set_xticks(np.hstack(xticks))
ax.set_xticklabels(())

xtick_labels = ["1", "2", "3", "4"]
ax.set_xticklabels(xtick_labels * 10, rotation=0)
ax.set_xlabel("rank $\\ell$ involved in the tensor product", fontsize=9)

ax.set_ylabel("%RMSE", fontsize=9)
for i, x in enumerate(xlabels):
    ax.text(x=xticks[1::4][i], y=190, s=x, fontsize=9)
ax.set_ylim(6, 650)
ax.set_xlim(xticks[0] - 1, xticks[-1] + 1)
ax.text(0, 400, "oxygen", fontsize=9)
ax.grid(lw=0.5, alpha=0.5, which="both")
ax.tick_params(axis="both", labelsize=7)
ax.set_xlim(xticks[0] - 1, xticks[-1] + 1)